# Title Generator Using T5

References: [Here](https://shivanandroy.com/transformers-generating-arxiv-papers-title-from-abstracts/)

## Load Data

In [16]:
import json

data = '../dataset/arxiv-metadata-oai-snapshot.json'

def get_metadata():
    with open(data, 'r') as f:
        for line in f:
            yield line

In [17]:
metadata = get_metadata()

for paper in metadata:
    paper_dict = json.loads(paper)
    print('Title: {}\n\nAbstract: {}\nRef: {}'.format(paper_dict.get('title'), paper_dict.get('abstract'), paper_dict.get('journal-ref')))
    break

Title: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies

Abstract:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. The region of phase space is
specified in which the calculation is most reliable. Good agreement is
demonstrated with data from the Fermilab Tevatron, and predictions are made for
more detailed tests with CDF and DO data. Predictions are shown for
distributions of diphoton pairs produced at the energy of the Large Hadron
Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs
boson are contrasted with those produced from QCD processes at the LHC, showing
tha

Take last 5 years of ArXiv papers due to computational resource limits.

In [18]:
titles = []
abstracts = []
years = []
metadata = get_metadata()

for paper in metadata:
    paper_dict = json.loads(paper)
    ref = paper_dict.get('journal-ref')
    try:
        year = int(ref[-4:])
        if 2016 < year < 2021:
            years.append(year)
            titles.append(paper_dict.get('title'))
            abstracts.append(paper_dict.get('abstract'))
    except:
        pass

len(titles), len(abstracts), len(years)

(18566, 18566, 18566)

There are about 18K research papers published from 2016 to 2020.

### Convert Into DataFrames

In [19]:
import pandas as pd

papers = pd.DataFrame({
    'title': titles,
    'abstract': abstracts,
    'year': years
})

papers.head()

,title,abstract,year
0,On the Cohomological Derivation of Yang-Mills ...,We present a brief review of the cohomologic...,2017
1,Regularity of solutions of the isoperimetric p...,In this work we consider a question in the c...,2018
2,Asymptotic theory of least squares estimators ...,This paper considers the effect of least squ...,2017
3,"Teichm\""uller Structures and Dual Geometric Gi...",The Gibbs measure theory for smooth potentia...,2020
4,Distributional Schwarzschild Geometry from non...,In this paper we leave the neighborhood of t...,2018


### Split Train-Test Data

In [20]:
# Splitting data using the 80:20 training-testing ratio
eval_df = papers.sample(frac=0.2, random_state=673)
train_df = papers.drop(eval_df.index)

train_df.shape, eval_df.shape

((14853, 3), (3713, 3))

In [21]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_df)
eval_data = Dataset.from_pandas(eval_df)

In [22]:
train_data, eval_data

(Dataset({
     features: ['title', 'abstract', 'year', '__index_level_0__'],
     num_rows: 14853
 }),
 Dataset({
     features: ['title', 'abstract', 'year', '__index_level_0__'],
     num_rows: 3713
 }))

In [23]:
train_data = train_data.remove_columns('__index_level_0__')
eval_data = eval_data.remove_columns('__index_level_0__')

train_data, eval_data

(Dataset({
     features: ['title', 'abstract', 'year'],
     num_rows: 14853
 }),
 Dataset({
     features: ['title', 'abstract', 'year'],
     num_rows: 3713
 }))

In [24]:
train_data.shape, eval_data.shape

((14853, 3), (3713, 3))

## Preprocessing

### Tokenize Features

In [25]:
from transformers import AutoTokenizer

path = 'sshleifer/distilbart-cnn-12-6'

tokenizer = AutoTokenizer.from_pretrained(path)

loading configuration file https://huggingface.co/sshleifer/distilbart-cnn-12-6/resolve/main/config.json from cache at /home/liana/.cache/huggingface/transformers/adac95cf641be69365b3dd7fe00d4114b3c7c77fb0572931db31a92d4995053b.a50597c2c8b540e8d07e03ca4d58bf615a365f134fb10ca988f4f67881789178
Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "for

In [26]:
train_dataset = train_data.map(lambda x: tokenizer(train_data['abstract'], padding='max_length', truncation=True), remove_columns=train_data.column_names, batched=True)
eval_dataset = eval_data.map(lambda x: tokenizer(eval_data['abstract'], padding='max_length', truncation=True), remove_columns=eval_data.column_names, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

## Model

In [27]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(path)

loading configuration file https://huggingface.co/sshleifer/distilbart-cnn-12-6/resolve/main/config.json from cache at /home/liana/.cache/huggingface/transformers/adac95cf641be69365b3dd7fe00d4114b3c7c77fb0572931db31a92d4995053b.a50597c2c8b540e8d07e03ca4d58bf615a365f134fb10ca988f4f67881789178
Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "for

## Training

In [28]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='trainer' + path,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [30]:
trainer.train()

***** Running training *****
  Num examples = 222795
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 55700
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## Training

The `simpletransformers` library is used to train the `T5 model`.

In [11]:
# # Adding input and target columns
# papers = papers[['title', 'abstract']]
# papers.columns = ['target_text', 'input_text']

# # Adding prefix columns
# papers['prefix'] = 'summarize'

# # Splitting data using the 80:20 training-testing ratio
# eval_df = papers.sample(frac=0.2, random_state=673)
# train_df = papers.drop(eval_df.index)

# train_df.shape, eval_df.shape

In [12]:
# import logging

# from simpletransformers.t5 import T5Model

# # Setting logging information
# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger('transformers')
# transformers_logger.setLevel(logging.WARNING)

# # Training parameters
# params = {
#     'reprocess_input_data': True,
#     'overwrite_output_dir': True,
#     'max_seq_length': 512,
#     'train_batch_size': 16,
#     'num_train_epochs': 4,
#     'best_model_dir': '../models/title-generator',
#     'fp16': False
# }

# # Creating model
# model = T5Model(model_type='t5', model_name='t5-small', args=params, use_cuda=False)

# # Training model
# model.train_model(train_df)

# # Evaluating results
# results = model.eval_model(eval_df)

# print(results)